# Load Data and preprocess

In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))

In [2]:
#pip install ucimlrepo

In [3]:
from src.data.load_data import load_input_data
from src.data.load_data import preprocess_data

In [4]:
X_train, X_test, y_train, y_test, metadata, variables, datasetdf = load_input_data()

In [5]:
num_cols = ['Attribute2', 'Attribute5', 'Attribute8', 'Attribute11',
            'Attribute13', 'Attribute16', 'Attribute18']

X_train_proc, X_test_proc = preprocess_data(X_train, X_test, num_cols)
#pd.set_option('display.max_columns', None)
#print(X_train.head())

# Train model

In [6]:
from src.model.train_model import train_classifier

In [7]:
clf = train_classifier(X_train_proc,y_train,n_estimators=100, random_state=42)

# Test model

In [8]:
from src.model.test_model import test_classifier_prediction

In [9]:
y_pred,classification_report,accuracy_score = test_classifier_prediction(clf,X_test_proc,y_test)

In [10]:
print(accuracy_score)

0.76


In [11]:
print(classification_report)

              precision    recall  f1-score   support

           1       0.76      0.95      0.85       210
           2       0.74      0.31      0.44        90

    accuracy                           0.76       300
   macro avg       0.75      0.63      0.64       300
weighted avg       0.76      0.76      0.72       300



### User constraints / Ontology constraints

In [12]:
from src.ontology.generate_rules import summarize_features
from src.ontology.generate_rules import extract_statistical_rules
from src.ontology.generate_rules import top_features
from src.ontology.generate_rules import combined_features

In [13]:
X_train_positive = X_train[y_train == 1]
X_train_negative = X_train[y_train == 2]

In [14]:
positive_profile = summarize_features(X_train_positive)

In [15]:
negative_profile = summarize_features(X_train_negative)

In [16]:
p_statistical = extract_statistical_rules(X_train_positive)

In [18]:
p_statistical

{'Attribute2': (6.0, 36.0),
 'Attribute5': (929.9, 6473.9),
 'Attribute8': (1.0, 4.0),
 'Attribute10': 'A101',
 'Attribute11': (1.0, 4.0),
 'Attribute13': (24.0, 51.0),
 'Attribute14': 'A143',
 'Attribute15': 'A152',
 'Attribute16': (1.0, 2.0),
 'Attribute18': (1.0, 2.0),
 'Attribute20': 'A201'}

In [64]:
p_top_features = top_features(clf, X_train_proc, y_train, n_repeats=5, random_state=42)

Top important features: ['Attribute1_A14', 'Attribute2', 'Attribute5', 'Attribute13', 'Attribute6_A65', 'Attribute14_A143']


In [ ]:
p_top_features

In [68]:
p = combined_features(p_top_features,p_statistical)

In [69]:
p_combined

{'Attribute2': (6.0, 36.0),
 'Attribute5': (929.9, 6473.9),
 'Attribute8': (1.0, 4.0),
 'Attribute10': 'A101',
 'Attribute11': (1.0, 4.0),
 'Attribute13': (24.0, 51.0),
 'Attribute14': 'A143',
 'Attribute16': (1.0, 2.0),
 'Attribute18': (1.0, 2.0),
 'Attribute20': 'A201',
 'Attribute1': 'A14',
 'Attribute6': 'A65'}

### Final outputs of Input section of pipeline
For experimentation, here are some values

In [13]:
from src.utilities.desired_space import desired_space
x= X_test[0] #test instance
p= {
    'Attribute5':(10000, 30000) #credit amount
}#perturbation map
desired_space= desired_space(2, datasetdf) #all the feature vectors with output=2 #not preprocessed! Potential issue from authors!
cat_f= X_train.select_dtypes(include=['bool']) #categorical features
num_f= X_train.select_dtypes(include=['int64', 'float64']) #numerical features
protect_f= ['Attribute9_A92'  'Attribute9_A93'  'Attribute9_A94'] # Personal status and sex (X_train.head() shows only 3 values (columns) while there are 5) #protected features
features= X_train.columns() #all features
t= 2 #desired outcome
f= clf #black-box model for which we want to find CF
X= X_train #training data
step= {
    'Attribute5': 10 #credit amount steps
} #dictionary holding the feature distribution to be used for single feature perturbation method

KeyError: 0

### Derived variables

In [11]:
f2change = p.keys()

NameError: name 'p' is not defined

# UFCE

### Feature Selection: Mutual Information (MI)

In [2]:
from itertools import combinations
from src.utilities.mutual_information import mi_score
from src.utilities.mutual_information import MI
features = list(X_train.columns)
feature_pairs = list(combinations(features, 2))
print(feature_pairs[:5])
mi_list = MI(feature_pairs, X_train) #used all features available #original authors do over entire X but it is not preprocessed... Which i think is nessecary? 
# top 5 
print(mi_list[:5])

ModuleNotFoundError: No module named 'src'

### Nearest Neighbour: KD Tree

In [ ]:
from src.utilities.nearest_neighbours import FNN
nn = FNN(desired_space,x,50) #radius arbitrarily set; can experiment with how close we want it to be to x

### Subspace: Intervals

In [ ]:
from src.utilities.nearest_neighbours import intervals
subspace = intervals(nn,p,f2change,x)

### CF Generator: Single Feature